# Image Classification with a Multi-Layer Perceptron

**Notebook created in PyTorch by [Santi Pascual](https://github.com/santi-pdp) for the [UPC School](https://www.talent.upc.edu/ing/estudis/formacio/curs/310400/postgrau-artificial-intelligence-deep-learning/) (2019).**

Based on an original version in Keras created by [Miriam Bellver](https://imatge.upc.edu/web/people/miriam-bellver) for the [Barcelona Technology School](https://barcelonatechnologyschool.com/master/master-in-big-data-solutions/) (BTS) in 2018, and updated by [Daniel Fojo](https://www.linkedin.com/in/daniel-fojo/) and [Xavier Giro](https://imatge.upc.edu/web/people/xavier-giro) in 2019.

The problem we are trying to solve here is to classify grayscale images of handwritten digits (28 pixels by 28 pixels), into their 10 categories (0 to 9). The dataset we will use is the MNIST dataset, a classic dataset in the machine learning community, which has been around for almost as long as the field itself and has been very intensively studied. It's a set of 60,000 training images, plus 10,000 test images, assembled by the National Institute of Standards and Technology (the NIST in MNIST) in the 1980s. You can think of "solving" MNIST as the "Hello World" of deep learning -- it's what you do to verify that your algorithms are working as expected. As you become a machine learning practitioner, you will see MNIST come up over and over again, in scientific papers, blog posts, and so on.

The MNIST dataset comes pre-loaded in Keras, in the form of a set of four Numpy arrays:


In [0]:
# Set the random seed 123 from numpy
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.seed.html
import numpy as np
np.random.seed(123)
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms


train_images and train_labels form the "training set", the data that the model will learn from. The model will then be tested on the "test set", test_images and test_labels. Our images are encoded as Numpy arrays, and the labels are simply an array of digits, ranging from 0 to 9. There is a one-to-one correspondence between the images and the labels.

### Defining the Hyper-parameters

We now define the hyperparameters that are going to be used throughout the notebook
to define the network, the data `batch_size`, the training `learning_rate`, and others.

In [0]:
# Let's define some hyper-parameters
hparams = {
    'batch_size':64,
    'num_epochs':10,
    'test_batch_size':64,
    'hidden_size':128,
    'num_classes':10,
    'num_inputs':784,
    'learning_rate':1e-3,
    'log_interval':100,
}

# we select to work on GPU if it is available in the machine, otherwise
# will run on CPU
hparams['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'

# whenever we send something to the selected device (X.to(device)) we already use
# either CPU or CUDA (GPU). Importantly...
# The .to() operation is in-place for nn.Module's, so network.to(device) suffices
# The .to() operation is NOT in.place for tensors, so we must assign the result
# to some tensor, like: X = X.to(device)

### Defining the PyTorch Dataset and the DataLoader

The PyTorch Dataset (https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) is an inheritable `class` that helps us defining what source of data do we have (image, audio, text, ...) and how to load it (overriding the `__getitem__` function). The MNIST dataset is easible accessible from it.

In [0]:
mnist_trainset = datasets.MNIST('data', train=True, download=True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ]))
mnist_testset = datasets.MNIST('data', train=False, 
                               transform=transforms.Compose([
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.1307,), (0.3081,))
                               ]))

train_loader = torch.utils.data.DataLoader(
    mnist_trainset,
    batch_size=hparams['batch_size'], 
    shuffle=True)

test_loader = torch.utils.data.DataLoader(
    mnist_testset,
    batch_size=hparams['test_batch_size'], 
    shuffle=False)

In [0]:
# We can retrieve a sample from the dataset by simply indexing it
img, label = mnist_trainset[0]
print('Img shape: ', img.shape)
print('Label: ', label)

# Similarly, we can sample a BATCH from the dataloader by running over its iterator
iter_ = iter(train_loader)
bimg, blabel = next(iter_)
print('Batch Img shape: ', bimg.shape)
print('Batch Label shape: ', blabel.shape)
print('The Batched tensors return a collection of {} grayscale images ({} channel, {} height pixels, {} width pixels)'.format(bimg.shape[0],
                                                                                                                              bimg.shape[1],
                                                                                                                              bimg.shape[2],
                                                                                                                              bimg.shape[3]))
print('In the case of the labels, we obtain {} batched integers, one per image'.format(blabel.shape[0]))

And now let's look at the kind of images we are dealing with:


In [0]:
import matplotlib.pyplot as plt
import random

# Definition of a Python function that plots a NxN grid of images contained
# in the images array
def plot_samples(images,N=5):

    '''
    Plots N**2 randomly selected images from training data in a NxN grid
    '''
    
    # Randomly select NxN images and save them in ps
    ps = random.sample(range(0,images.shape[0]), N**2)

    # Allocates figure f divided in subplots contained in an NxN axarr
    # https://matplotlib.org/3.1.0/api/_as_gen/matplotlib.pyplot.subplots.html
    f, axarr = plt.subplots(N, N)

    # Index for the images in ps to be plotted
    p = 0
    
    # Scan the NxN positions of the grid
    for i in range(N):
        for j in range(N):
          
            # Load the image pointed by p
            im = images[ps[p]]
          
            # If images are encoded in grayscale 
            # (a tensor of 3 dimensions: width x height x luma)...
            if len(images.shape) == 3:
              # ...specify the colormap as grayscale
              # https://matplotlib.org/3.1.0/api/_as_gen/matplotlib.pyplot.imshow.html
              axarr[i,j].imshow(im,cmap='gray')            
            else:              
              # ...no need to specify any color map
              axarr[i,j].imshow(im)
              
            # Remove axis
            axarr[i,j].axis('off')
            
            # Point to the next image from the random selection
            p+=1
    # Show the plotted figure         
    plt.show()

# convert the dataloader output tensors from the previous cell to numpy arrays
# The channel dimension has to be squeezed in order for matplotlib to work
# with grayscale images
img = bimg.squeeze(1).data.numpy()
plot_samples(img)

### BAAAAM, we've got some numbers there

# Training a Multi-Layer Perceptron (MLP)

Now that we have the dataset loaded and prepared, let's get some deep stuff spinning. 

Our workflow will be as follow: first we will train our neural network with the training data, loaded from the constructed `train_loader`. The network will then learn to associate images and labels. Finally, we will ask the network to produce predictions for `test_loader` images, and we will verify if these predictions match the labels from `test_loader`.

### Exercise 1

For the time being, we will use a very simple network. It consists of a sequence of two `nn.Linear` layers, which are densely-connected (also called "fully-connected") neural layers. The last layer is a 10-way `nn.LogSoftmax` layer, which means it will return an array of 10 log-probability scores. Each score will be the probability that the current digit image belongs to one of our 10 digit classes. Please fill in the network definition below.

In [0]:
# Define a variable 'network' by instantiating a PyTorch sequential model
# https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential

# Add a (1) nn.Linear hidden layer with 128 neurons,, (2) a nn.ReLU, (3) the output nn.Linear and,
# the (4) output nn.LogSoftmax
# NOTE: Consider the 'input_size', 'hidden_size', and 'num_classes' parameters 
# defined above as hyper-params

#network=TODO

Now we can check which is the architecture of the network, and the number of parameters of each layer with the following helper function:

In [0]:
def get_nn_nparams(net):
  """ https://discuss.pytorch.org/t/how-do-i-check-the-number-of-parameters-of-a-model/4325/6 """
  pp=0
  for p in list(net.parameters()):
      nn=1
      for s in list(p.size()):
          nn = nn*s
      pp += nn
  return pp
  
print(network)
print('Num params: ', get_nn_nparams(network))

### Exercise 2

To make our network ready for training, we need to pick three more things:

*    **A loss function**: this is how the network will be able to measure how good a job works on its training data, and thus how it will be able to steer itself in the right direction. **Check the [PyTorch documentation](https://pytorch.org/docs/stable/nn.functional.html#nll-loss) for the negative log likelihood loss for multi-class classification from the functional API.**
*   **An optimizer**: this is the mechanism through which the network will update itself based on the data it sees and its loss function. **Check the PyTorch documentation and find the RMSprop optimizer to use it.**
*   **Metrics to monitor during training and testing**. Here we will only care about accuracy (the fraction of the images that were correctly classified). **Define the accuracy function to return, for a batch, the count of correct predictions (hence same prediction as the label).**





In [0]:
# Look for the optimizer within the documentation below
import torch.optim as optim

# Optimizer: RMS Prop (use the hparams['learning_rate'] previously defined)
# https://pytorch.org/docs/stable/optim.html#torch.optim.RMSprop
optimizer=TODO

# Negative Log Likelihood (NLL) Loss criterion from functional API
# https://pytorch.org/docs/stable/nn.functional.html#nll-loss
criterion=TODO

# Define the Accuracy metric in the function below by:
  # (1) obtain the maximum for each predicted element in the batch to get the class (it is the maximum index of the num_classes array per batch sample) (look at torch.argmax in the PyTorch documentation)
  # (2) compare the predicted class index with the index in its corresponding neighbor within label_batch 
  # (3) sum up the number of affirmative comparisons and return the summation
def correct_predictions(predicted_batch, label_batch):
  pred = predicted_batch.argmax(dim=1, keepdim=True) # get the index of the max log-probability
  acum = pred.eq(label_batch.view_as(pred)).sum().item()
  return acum

### Exercise 3

Next, the training and test functions that iterate for an epoch are defined. They are missing a key ingredient to make the `data` tensors, which include images, compatible with the fully connected input of size `num_inputs` of the network. Please fix this in both cases, remembering the use of `view()` in the previous lab.

In [0]:
def train_epoch(train_loader, network, optimizer, criterion, hparams):
  # Activate the train=True flag inside the model
  network.train()
  device = hparams['device']
  avg_loss = None
  avg_weight = 0.1
  for batch_idx, (data, target) in enumerate(train_loader):
      data, target = data.to(device), target.to(device)
      optimizer.zero_grad()
      # TO DO
      data = TO DO
      """ SOLUCIO: data = data.view(data.shape[0], -1) """
      output = network(data)
      loss = criterion(output, target)
      loss.backward()
      if avg_loss:
        avg_loss = avg_weight * loss.item() + (1 - avg_weight) * avg_loss
      else:
        avg_loss = loss.item()
      optimizer.step()
      if batch_idx % hparams['log_interval'] == 0:
          print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
              epoch, batch_idx * len(data), len(train_loader.dataset),
              100. * batch_idx / len(train_loader), loss.item()))
  return avg_loss

In [0]:
def test_epoch(test_loader, network, hparams):
    network.eval()
    device = hparams['device']
    test_loss = 0
    acc = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            data = data.view(data.shape[0], -1)
            output = network(data)
            test_loss += criterion(output, target, reduction='sum').item() # sum up batch loss
            # compute number of correct predictions in the batch
            acc += correct_predictions(output, target)
    # Average acc across all correct predictions batches now
    test_loss /= len(test_loader.dataset)
    test_acc = 100. * acc / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, acc, len(test_loader.dataset), test_acc,
        ))
    return test_loss, test_acc

### Exercise 4

Now that the `network` and the training and test functions are good to go epoch by epoch, build the loop to make as many as `hparams['num_epochs']` epochs by alternating the train and test phases. Check what these functions return and store the resulting values in lists that will be used to plot the results in the later cell.

In [0]:
# Init lists to save the evolution of the training & test losses/accuracy.
train_losses = []
test_losses = []
test_accs = []

for epoch in range(1, hparams['num_epochs'] + 1):
  train_losses.append(train_epoch(train_loader, network, optimizer, criterion, hparams))

  # TO DO
  # test_loss,test_accuracy=TODO
  
  test_losses.append(test_loss)
  test_accs.append(test_accuracy)

In [0]:
plt.figure(figsize=(10, 8))
plt.subplot(2,1,1)
plt.xlabel('Epoch')
plt.ylabel('NLLLoss')
plt.plot(train_losses, label='train')
plt.plot(test_losses, label='test')
plt.legend()
plt.subplot(2,1,2)
plt.xlabel('Epoch')
plt.ylabel('Test Accuracy [%]')
plt.plot(test_accs)

#### References

[1] https://github.com/pytorch/examples/blob/master/mnist/main.py